# Check GPU status

Make surre to use : GPU runtime mode (Runtime->Change Runtime type -> python3 + GPU
)



In [ ]:
# Check nvidia and nvcc cuda compiler

!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Tue Feb 16 16:41:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Mount Goolge Drive

In [ ]:
# link to google drive

from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [ ]:
#check that Gdrive is mounted

!ls '/content/gdrive/My Drive/Colab Notebooks'

'16_nlp_with_rnns_and_attention (1).ipynb'
 16_nlp_with_rnns_and_attention.ipynb
 Bert-GPU.ipynb
 Chap1_Introduction_PyTorch.ipynb
 Chap3_NN_Components.ipynb
 Chap4_MLP_CNN.ipynb
 Chap5.ipynb
'Chap8_Encoder–decoder (1).ipynb'
 Chap8_Encoder–decoder.ipynb
'ColabEnv (1).ipynb'
 ColabEnv.ipynb
'Copy of Welcome To Colaboratory'
'Copy of Welcome To Colaboratory (1)'
'Copy of Welcome To Colaboratory (2)'
 CUDA_GPU_server.ipynb
 data
 model_storage
 NLP-LOTClass.ipynb
 pyTorchApexipynb.ipynb
 Transformers.ipynb
 Untitled
 Untitled0.ipynb
 Untitled1.ipynb


#Setup SSH port forwarding

In [ ]:
#1 - setup ssh/user 


#Generate a random root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(30))


#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

print("username: root")
print("password: ", password)

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

username: root
password:  nD4zg6tjR8EZ1eMmQI39x0l0QmQrN9


In [ ]:
# 2 - Download Ngrok

! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

In [ ]:
# 3 - setup Ngrok - authtoken

#Ask token
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

Get your authtoken from https://dashboard.ngrok.com/auth
··········


Congratulations, you are ready to go. 
On Ngrok interface https://dashboard.ngrok.com/status you'll find the tcp address and the port 

connect using the following : 

```
ssh root@0.tcp.ngrok.io -p [ngrok_port]

> then enter the password generated previously

ssh root@2.tcp.ngrok.io -p 12024
ssh root@4.tcp.ngrok.io -p 10942
ssh root@2.tcp.ngrok.io -p 13462
```

In [ ]:
# When done, kill Ngrok

!kill $(ps aux | grep './ngrok' | awk '{print $2}')

In [ ]:
# Mount Google Drive and make some folders for vscode
from google.colab import drive
drive.mount('/googledrive')
! mkdir -p /googledrive/My\ Drive/colabdrive
! mkdir -p /googledrive/My\ Drive/colabdrive/root/.local/share/code-server
! ln -s /googledrive/My\ Drive/colabdrive /
! ln -s /googledrive/My\ Drive/colabdrive/root/.local/share/code-server /root/.local/share/

Drive already mounted at /googledrive; to attempt to forcibly remount, call drive.mount("/googledrive", force_remount=True).
ln: failed to create symbolic link '/colabdrive': File exists
ln: failed to create symbolic link '/root/.local/share/code-server': File exists


In [ ]:
! curl -fsSL https://code-server.dev/install.sh | sh > /dev/null
! code-server --bind-addr 127.0.0.1:9998 --auth none &


[2021-02-16T16:44:49.004Z] info  code-server 3.9.0 fc6d123da59a4e5a675ac8e080f66e032ba01a1b
[2021-02-16T16:44:49.005Z] info  Using user-data-dir ~/.local/share/code-server
[2021-02-16T16:44:49.021Z] info  Using config file ~/.config/code-server/config.yaml
[2021-02-16T16:44:49.021Z] info  HTTP server listening on http://127.0.0.1:9998 
[2021-02-16T16:44:49.021Z] info    - Authentication is disabled 
[2021-02-16T16:44:49.021Z] info    - Not serving HTTPS 
[2021-02-16T18:24:44.619Z] error vscode Handshake timed out {"token":"dfe2bfe5-d6dc-4483-a151-459449212814"}
events.js:292
      throw er; // Unhandled 'error' event
      ^

Error: write EPIPE
    at WriteWrap.onWriteComplete [as oncomplete] (internal/stream_base_commons.js:92:16)
Emitted 'error' event on Socket instance at:
    at errorOrDestroy (internal/streams/destroy.js:108:12)
    at onwriteError (_stream_writable.js:424:5)
    at onwrite (_stream_writable.js:445:5)
    at internal/streams/destroy.js:50:7
    at Socket._destroy 

In [ ]:
ssh -L 9999:localhost:9999 root@2.tcp.ngrok.io -p 12024
ssh -L 9998:localhost:9998 root@4.tcp.ngrok.io -p 10942
ssh -L 9998:localhost:9998 root@2.tcp.ngrok.io -p 13462

In [ ]:
https://towardsdatascience.com/colab-free-gpu-ssh-visual-studio-code-server-36fe1d3c5243

https://github.com/yumeng5/LOTClass/issues/2


```
export CUDA_DEVICE_ORDER=PCI_BUS_ID
export CUDA_VISIBLE_DEVICES=0,1

DATASET=agnews
LABEL_NAME_FILE=label_names.txt
TRAIN_CORPUS=train.txt
TEST_CORPUS=test.txt
TEST_LABEL=test_labels.txt
MAX_LEN=200
TRAIN_BATCH=32
ACCUM_STEP=4
EVAL_BATCH=128
GPUS=1
MCP_EPOCH=3
SELF_TRAIN_EPOCH=1

python3 src/train.py --dataset_dir datasets/${DATASET}/ --label_names_file ${LABEL_NAME_FILE} \
                    --train_file ${TRAIN_CORPUS} \
                    --test_file ${TEST_CORPUS} --test_label_file ${TEST_LABEL} \
                    --max_len ${MAX_LEN} \
                    --train_batch_size ${TRAIN_BATCH} --accum_steps ${ACCUM_STEP} --eval_batch_size ${EVAL_BATCH} \
                    --gpus ${GPUS} \
                    --mcp_epochs ${MCP_EPOCH} --self_train_epochs ${SELF_TRAIN_EPOCH} \
```

pip3 install "transformers==3.3.1"

On Linux, NCCL and torch.distributed will be enabled by default. On MacOs, with PyTorch 1.3.1+, you need to conda install libuv and pkg-config explicitly set USE_DISTRIBUTED=1 when compiling from source. For Windows, torch.distributed is not enabled yet.